In [3]:
import numpy as np
import math
from scipy.integrate import fixed_quad

# Composite Trapezoidal Rule

def composite_trapezoidal(a, b, f, N):
    h = (b - a) / N
    x = [a + i * h for i in range(N + 1)]
    result = 0.5 * (f(x[0]) + f(x[-1])) + sum(f(xi) for xi in x[1:-1])
    return h * result

# Composite Simpson's Rule

def composite_simpson(a, b, f, N):
    if N % 2 == 1:
        raise ValueError("N must be an even number for Simpson's Rule.")
    h = (b - a) / N
    x = [a + i * h for i in range(N + 1)]
    result = f(x[0]) + f(x[-1])
    result += 4 * sum(f(x[i]) for i in range(1, N, 2))  # Odd indices
    result += 2 * sum(f(x[i]) for i in range(2, N - 1, 2))  # Even indices
    return h / 3 * result

# Adaptive Trapezoidal Quadrature

def adaptive_trapezoidal(f, a, b, tol, max_depth=10, interval_count=None):
    if interval_count is None:
        interval_count = {"count": 0}

    def recurse(a, b, tol, depth):
        interval_count["count"] += 1
        N = 1
        I1 = composite_trapezoidal(a, b, f, N)
        I2 = composite_trapezoidal(a, b, f, N * 2)
        if abs(I2 - I1) < 3 * tol or depth >= max_depth:
            return I2
        mid = (a + b) / 2
        return recurse(a, mid, tol / 2, depth + 1) + recurse(mid, b, tol / 2, depth + 1)

    result = recurse(a, b, tol, 0)
    return result, interval_count["count"]

# Adaptive Simpson's Quadrature

def adaptive_simpson(f, a, b, tol, max_depth=10, interval_count=None):
    if interval_count is None:
        interval_count = {"count": 0}

    def recurse(a, b, tol, depth):
        interval_count["count"] += 1
        N = 2
        I1 = composite_simpson(a, b, f, N)
        I2 = composite_simpson(a, b, f, N * 2)
        if abs(I2 - I1) < 15 * tol or depth >= max_depth:
            return I2
        mid = (a + b) / 2
        return recurse(a, mid, tol / 2, depth + 1) + recurse(mid, b, tol / 2, depth + 1)

    result = recurse(a, b, tol, 0)
    return result, interval_count["count"]

# Adaptive Gaussian Quadrature

def adaptive_gaussian(f, a, b, tol, max_depth=10, interval_count=None):
    if interval_count is None:
        interval_count = {"count": 0}

    def recurse(a, b, tol, depth):
        interval_count["count"] += 1
        I1, _ = fixed_quad(f, a, b, n=5)
        if depth >= max_depth:
            return I1
        mid = (a + b) / 2
        return recurse(a, mid, tol / 2, depth + 1) + recurse(mid, b, tol / 2, depth + 1)

    result = recurse(a, b, tol, 0)
    return result, interval_count["count"]

# Test function and interval
def target_function(x):
    return np.sin(1 / x)

a = 0.1
b = 2.0
tol = 1e-3

# Results
trap_result, trap_intervals = adaptive_trapezoidal(target_function, a, b, tol)
simp_result, simp_intervals = adaptive_simpson(target_function, a, b, tol)
gauss_result, gauss_intervals = adaptive_gaussian(target_function, a, b, tol)

# Output results
print("Adaptive Trapezoidal Result:", trap_result)
print("Adaptive Simpson's Result:", simp_result)
print("Adaptive Gaussian Result:", gauss_result)
print("Intervals Used:")
print("Trapezoidal:", trap_intervals)
print("Simpson's:", simp_intervals)
print("Gaussian:", gauss_intervals)


Adaptive Trapezoidal Result: 1.1458077570712315
Adaptive Simpson's Result: 1.145499275751985
Adaptive Gaussian Result: 1.1455808340995004
Intervals Used:
Trapezoidal: 133
Simpson's: 15
Gaussian: 2047
